# Value-at-Risk (VaR) and Expected Shortfall (ES) using Historical Simulation, Variance-Covariance and Monte Carlo Simulation methods

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from scipy.stats import norm

In [2]:
def get_data_yf(tickers, start_date, end_date, interval='1d'):
    # Download the data
    df = pd.DataFrame(yf.download(tickers, start=start_date, end=end_date, interval = interval)['Adj Close'])
    return df

def historical_var(returns,confidence_level):
    losses = -returns
    position = int(np.ceil(len(losses)*(1-confidence_level)-1))
    sorted_array = np.sort(losses.values)[::-1]
    return [sorted_array[position], sorted_array]

def historical_es(returns, confidence_level):
    losses = -returns
    position = int(np.ceil(len(losses)*(1-confidence_level)-1))
    sorted_array = np.sort(losses.values)[::-1]
    return sorted_array[:position].mean()

def portfolio_stats(mu, cov, weights):
    mean_returns = mu.values
    cov_matrix = cov.values
    portfolio_mean = np.dot(weights, mean_returns)
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    
    return [portfolio_mean, portfolio_std_dev]

def parametric_var(portfolio_mean, portfolio_std, confidence_level):
    Z_alpha = norm.ppf(confidence_level)
    VaR = - portfolio_mean + Z_alpha * portfolio_std_dev
    ES = - portfolio_mean + (norm.pdf(Z_alpha) * portfolio_std_dev) / (1 - confidence_level)
    return [VaR, ES]

def mc_var(returns,confidence_level):
    losses = -returns
    position = int(np.ceil(len(losses)*(1-confidence_level)-1))
    sorted_array = np.sort(losses)[::-1]
    return sorted_array[position]

def mc_es(returns, confidence_level):
    losses = -returns
    position = int(np.ceil(len(losses)*(1-confidence_level)-1))
    sorted_array = np.sort(losses)[::-1]
    return sorted_array[:position].mean()

def monteCarlo_simul(S0,r, T, N, number_of_stocks, sigma, weights, m = 100):
    dt = T/100
    simulated_stock_prices = np.zeros((num_stocks,N))
    
    for i in range(0,N):
        St = np.zeros((num_stocks,m+1))
        St[:,0] = S0
        for j in range(1,m+1):
            dW = np.random.normal(0, np.sqrt(dt), size=num_stocks)
            # Update stock prices using the Black-Scholes formula
            St[:, j] = St[:, j-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW)
        simulated_stock_prices[:,i] = St[:,-1]
    simulated_stock_prices_df = pd.DataFrame(simulated_stock_prices, index = data_df.columns)
    simulated_stock_returns = simulated_stock_prices - S0.values.reshape(-1,1)
    
    simulated_stock_returns_pct = simulated_stock_returns/S0.values.reshape(-1,1)
    simulated_portfolio_returns = weights.reshape(1,-1)@simulated_stock_returns_pct
    simulated_portfolio_returns = np.squeeze(simulated_portfolio_returns)
    
    return [simulated_stock_prices_df, simulated_portfolio_returns]

In [3]:
# Selecting 10 stocks for our portfolio
tickers = ['NVDA','TSLA','GOOG','META','AAPL','VKTX','AMD','MSFT','AMZN','IAU','SBUX']
start_date = datetime.strptime("2022-03-31","%Y-%m-%d")
end_date = datetime.strptime("2024-04-01","%Y-%m-%d")

In [4]:
data_df = get_data_yf(tickers, start_date, end_date)
data_df

[*********************100%%**********************]  11 of 11 completed


Ticker,AAPL,AMD,AMZN,GOOG,IAU,META,MSFT,NVDA,SBUX,TSLA,VKTX
Date,,,,,,,,,,,
2022-03-31,172.637146,109.339996,162.997498,139.649506,36.830002,222.124329,302.829315,272.517822,86.950325,359.200012,3.000000
2022-04-01,172.340546,108.190002,163.559998,140.699997,36.540001,224.611694,303.919617,266.785034,87.447342,361.529999,3.130000
2022-04-04,176.423874,110.529999,168.346497,143.642502,36.730000,233.642105,309.370941,273.256927,84.197571,381.816681,3.150000
2022-04-05,173.082077,106.820000,164.054993,141.063004,36.490002,231.594269,305.353668,258.984863,80.402992,363.753326,3.060000
2022-04-06,169.888550,103.669998,158.755997,137.175995,36.580002,223.063324,294.175903,243.763931,79.676582,348.586670,3.050000
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,172.279999,179.649994,178.869995,151.770004,40.930000,509.579987,428.739990,942.890015,90.709999,170.830002,69.550003
2024-03-25,170.850006,178.630005,179.710007,151.149994,41.049999,503.019989,422.859985,950.020020,90.669998,172.630005,69.190002
2024-03-26,169.710007,177.869995,178.300003,151.699997,41.200001,495.890015,421.649994,925.609985,90.360001,177.669998,80.830002


In [6]:
returns_data = pd.concat([data_df, np.log(data_df).diff()], axis = 1)
returns_data = returns_data.dropna()
num_stocks = data_df.shape[1]

new_columns = data_df.columns.to_list()
ret_columns = []
for i in data_df.columns.to_list():
    new_columns.append(i + ' Returns')
    ret_columns.append(i + ' Returns')

returns_data.columns = new_columns
new_columns = []

for i in range(data_df.shape[1]):
    new_columns.append(returns_data.columns[i])
    new_columns.append(returns_data.columns[i+11])

returns_data = returns_data[new_columns]

# Assign equal weights to each component
returns_data['Daily Portfolio Returns'] = returns_data[ret_columns].mean(axis = 1)
returns_data

returns_data.to_excel('Portfolio_returns_data.xlsx')

## VaR and ES using Historical Simulation Approach

In [7]:
returns_data_df = pd.read_excel('Portfolio_returns_data.xlsx')
returns_data_df.set_index(returns_data_df.columns[0], inplace = True)
returns_data_df

,AAPL,AAPL Returns,AMD,AMD Returns,AMZN,AMZN Returns,GOOG,GOOG Returns,IAU,IAU Returns,...,MSFT Returns,NVDA,NVDA Returns,SBUX,SBUX Returns,TSLA,TSLA Returns,VKTX,VKTX Returns,Daily Portfolio Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-01,172.340546,-0.001720,108.190002,-0.010573,163.559998,0.003445,140.699997,0.007494,36.540001,-0.007905,...,0.003594,266.785034,-0.021261,87.447342,0.005700,361.529999,0.006466,3.130000,0.042421,0.003527
2022-04-04,176.423874,0.023417,110.529999,0.021398,168.346497,0.028844,143.642502,0.020698,36.730000,0.005186,...,0.017778,273.256927,0.023969,84.197571,-0.037871,381.816681,0.054596,3.150000,0.006369,0.018527
2022-04-05,173.082077,-0.019124,106.820000,-0.034142,164.054993,-0.025823,141.063004,-0.018121,36.490002,-0.006556,...,-0.013070,258.984863,-0.053643,80.402992,-0.046115,363.753326,-0.048465,3.060000,-0.028988,-0.027532
2022-04-06,169.888550,-0.018623,103.669998,-0.029932,158.755997,-0.032833,137.175995,-0.027942,36.580002,0.002463,...,-0.037293,243.763931,-0.060569,79.676582,-0.009076,348.586670,-0.042589,3.050000,-0.003273,-0.027018
2022-04-07,170.195038,0.001802,103.720001,0.000482,157.784500,-0.006138,136.464996,-0.005197,36.700001,0.003275,...,0.006225,241.776443,-0.008187,79.447182,-0.002883,352.420013,0.010937,3.000000,-0.016529,-0.001617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,172.279999,0.005296,179.649994,0.005414,178.869995,0.004033,151.770004,0.020166,40.930000,-0.008030,...,-0.001468,942.890015,0.030736,90.709999,-0.010419,170.830002,-0.011582,69.550003,-0.014842,0.002080
2024-03-25,170.850006,-0.008335,178.630005,-0.005694,179.710007,0.004685,151.149994,-0.004094,41.049999,0.002928,...,-0.013810,950.020020,0.007533,90.669998,-0.000441,172.630005,0.010482,69.190002,-0.005190,-0.002263
2024-03-26,169.710007,-0.006695,177.869995,-0.004264,178.300003,-0.007877,151.699997,0.003632,41.200001,0.003647,...,-0.002866,925.609985,-0.026030,90.360001,-0.003425,177.669998,0.028777,80.830002,0.155492,0.011465


In [8]:
ptf_returns_series = returns_data_df['Daily Portfolio Returns']
var_historical_value, sorted_losses = historical_var(ptf_returns_series, .992)
es_historical_value = historical_es(ptf_returns_series, .992)
print('Historical VaR in percentage of portfolio is ', round(var_historical_value*100,2), '%')
print('Historical VaR assuming $1,000,000 portfolio is ', f'${format(round(var_historical_value*1000000),",")}')
print('\n')

print('Historical ES in percentage of portfolio is ', round(es_historical_value*100,2), '%')
print('Historical ES assuming $1,000,000 portfolio is ', f'${format(round(es_historical_value*1000000),",")}')

Historical VaR in percentage of portfolio is  5.19 %
Historical VaR assuming $1,000,000 portfolio is  $51,905


Historical ES in percentage of portfolio is  5.53 %
Historical ES assuming $1,000,000 portfolio is  $55,266


## VaR and ES using Variance-Covariance Method

In [9]:
returns_data_df_backup = returns_data_df.copy()
parametric_returns_df = returns_data_df[returns_data_df.columns[1::2]]

# Calculate mean, standard deviation and covariance of stock returns
mu_stocks = parametric_returns_df.mean()
std_stocks = parametric_returns_df.std(ddof = 0)
cov_stocks = parametric_returns_df.cov()

# Portfolio weights (assume equal weighting for simplicity)
weights = np.array([1/parametric_returns_df.shape[1]] * parametric_returns_df.shape[1])

# Calculate portfolio mean and standard devaition
portfolio_mean, portfolio_std_dev = portfolio_stats(mu_stocks,cov_stocks, weights)

# Calculate parametric VaR and ES assuming normal distribution of returns
param_var, param_es = parametric_var(portfolio_mean, portfolio_std_dev, confidence_level = 0.95)

print('Parametric VaR in percentage of portfolio is ', round(param_var*100,2), '%')
print('Parametric VaR assuming $1,000,000 portfolio is ', f'${format(round(param_var*1000000),",")}')
print('\n')

print('Parametric ES in percentage of portfolio is ', round(param_es*100,2), '%')
print('Parametric ES assuming $1,000,000 portfolio is ', f'${format(round(param_es*1000000),",")}')

Parametric VaR in percentage of portfolio is  2.97 %
Parametric VaR assuming $1,000,000 portfolio is  $29,652


Parametric ES in percentage of portfolio is  3.75 %
Parametric ES assuming $1,000,000 portfolio is  $37,451


## VaR and ES calculation using Monte Carlo simulations

Let the underlying price have a risk-neutral SDE given by:

$$
\frac{dS_t}{S_t} = r \, dt + \sigma \, dW_t
$$

where:
- $S_t$ is the stock price at time $t$.
- $r$ is the risk-free rate.
- $\sigma$ is the volatility of the stock's return.
- $dW_t$ is the increment of a Wiener process (or Brownian motion), representing the random component.


In [12]:
std_stocks = returns_data_df[returns_data_df.columns[1::2]].std(ddof = 0)
S0 = data_df.iloc[-1,:]

# Current risk free rate 10 year treasury rate on April 2, 2024
r = 0.0436
# Time interval is 1 day
T = 1/365

# Dividing 1 day into 100 intervals
m = 100

# Monte Carlo sample size
N = 5000


# Annualize stocks standard deviation
sigma = std_stocks.values*np.sqrt(252)

# Simulate prices and portfolio returns 
simulated_stock_prices_df, simulated_portfolio_returns = monteCarlo_simul(S0,r, T, N, num_stocks, sigma, weights, m = 100)

# Calculating montecarlo VaR and ES from simulated returns
monteCarlo_var = mc_var(simulated_portfolio_returns,0.95)
monteCarlo_es = mc_es(simulated_portfolio_returns, 0.95)

In [13]:
print('Monte Carlo VaR in percentage of portfolio is ', round(monteCarlo_var*100,2), '%')
print('Monte Carlo VaR assuming $1,000,000 portfolio is ', f'${format(round(monteCarlo_var*1000000),",")}')
print('\n')

print('Monte Carlo ES in percentage of portfolio is ', round(monteCarlo_es*100,2), '%')
print('Monte Carlo ES assuming $1,000,000 portfolio is ', f'${format(round(monteCarlo_es*1000000),",")}')

Monte Carlo VaR in percentage of portfolio is  1.29 %
Monte Carlo VaR assuming $1,000,000 portfolio is  $12,879


Monte Carlo ES in percentage of portfolio is  1.59 %
Monte Carlo ES assuming $1,000,000 portfolio is  $15,891
